In [11]:
# Imports
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.windows import from_bounds

In [14]:
# Locate all of the original UAV TIFs 

# Blaesedalen
bl1 = "../data/uav/M3Mexports/20230702-div128.tif"
bl2 = "../data/uav/M3Mexports/20230712-div128.tif"
bl3 = "../data/uav/M3Mexports/20230718-div128.tif"
bl4 = "../data/uav/M3Mexports/20230726-div128.tif"
bl5 = "../data/uav/MAIA-exports/20230726/20230726-div32768.tif"

# Kluane Low
kl1 = "../../data/uav/MAIA-exports/20220629/20220629-div32768.tif"
kl2 = "../../data/uav/MAIA-exports/20220705/20220705-div32768.tif"
kl3 = "../../data/uav/MAIA-exports/20220718/20220718-div32768.tif"
kl4 = "../../data/uav/MAIA-exports/20220814/20220814-div32768.tif"

# Kluane High
kh1 = "../data/uav/MAIA-exports/20220709/20220709-div32768.tif"
kh2 = "../data/uav/MAIA-exports/20220719/20220719-div32768.tif"
kh3 = "../data/uav/MAIA-exports/20220729/20220729-div32768.tif"
kh4 = "../data/uav/MAIA-exports/20220813/20220813-div32768.tif"

# List the UAV TIFs per location
blaesedalen = [bl1, bl2, bl3, bl4, bl5]
kluaneLow = [kl1, kl2, kl3, kl4]
kluaneHigh = [kh1, kh2, kh3, kh4]

# Concatenate to list of all plots
allUAV = [blaesedalen + kluaneLow + kluaneHigh]

In [13]:
# Clipping coordinates
klCoords = gpd.read_file("../../data/plot-coords/kluane-low.json") 

In [19]:
kh1[0:-4]

'../data/uav/MAIA-exports/20220709/20220709-div32768'

In [29]:
for tif in kluaneLow:
    with rasterio.open(tif) as src:
        # Perform clipping based on geojson geom
        for index, row in klCoords.iterrows():
            geom = row['geometry']
            print(geom)
            out_file = f"{tif[0:-4]}_clipped2.tif"

POLYGON ((-138.4177842009962 60.969333480490604, -138.41746769947537 60.96631710566089, -138.41079404621922 60.966274855157735, -138.41113914126714 60.969501214996, -138.4177842009962 60.969333480490604))
POLYGON ((-138.4177842009962 60.969333480490604, -138.41746769947537 60.96631710566089, -138.41079404621922 60.966274855157735, -138.41113914126714 60.969501214996, -138.4177842009962 60.969333480490604))
POLYGON ((-138.4177842009962 60.969333480490604, -138.41746769947537 60.96631710566089, -138.41079404621922 60.966274855157735, -138.41113914126714 60.969501214996, -138.4177842009962 60.969333480490604))
POLYGON ((-138.4177842009962 60.969333480490604, -138.41746769947537 60.96631710566089, -138.41079404621922 60.966274855157735, -138.41113914126714 60.969501214996, -138.4177842009962 60.969333480490604))


In [24]:
for tif in kluaneLow:
    with rasterio.open(tif) as src:
        # Perform clipping based on geojson geom
        for index, row in klCoords.iterrows():
            geom = row['geometry']
            out_file = f"{tif[0:-4]}_clipped2.tif"
            
            # Calculate the bounding box of the geom for efficient read
            clipped_data, _ = mask(src, [geom], crop=True)
            
            # Create new dataset for the clipped output
            with rasterio.open(out_file, 'w', driver = 'GTiff',
                              count = src.count, dtype = src.dtypes[0], crs = src.crs, transform = src.transform) as out_dst:
                # Read and write the clipped data
                for band in range(1, src.count + 1):
                    out_dst.write(src.read(band, window = window), band)

print('Clipping complete')
                
    

TypeError: Integer width and height are required.

In [27]:
for tif_file in kluaneLow:
    with rasterio.open(tif_file) as src:
        # Perform the clipping based on the GeoJSON geometries
        for index, row in klCoords.iterrows():
            geom = row['geometry']
            out_file = f'{tif_file[0:-4]}_clippedPy.tif'  # Change the output file name as needed

            # Clip the GeoTIFF using the polygon geometry
            clipped_data, out_transform = mask(src, [geom], crop=True)
            
            # Create a new dataset for the clipped output
            with rasterio.open(out_file, 'w', driver='GTiff', 
                               count=src.count,  # Number of bands
                               width=clipped_data.shape[2],  # Width of the clipped data
                               height=clipped_data.shape[1],  # Height of the clipped data
                               dtype=clipped_data.dtype, 
                               crs=src.crs, 
                               transform=out_transform) as out_dst:
                out_dst.write(clipped_data)

print("Clipping complete.")


Clipping complete.
